In [16]:
import time
import json
import pandas as pd
import numpy as np
import re
from itertools import compress
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [17]:
def json_to_csv(directory, fileNames, createSample=False):
    """
    json_to_csv: loops through specified JSON files and converts them to csv files.
                 option to also create a sample csv, which uses np.random.seed 9001 to create a sample dataset with 10% of the observations
    
                 pandas has a read_json function, but returns a 'Trailing data error' when working with these specific files
                 
                 
    Inputs: -directory of JSON files
            -list of JSON filenames
            -createSample flag
            
    """
    
    start = time.time()

    jsonData = []

    for fileName in fileNames:
        with open(directory + fileName,  encoding="utf8") as file:
            print('{0} opened'.format(fileName))
            for line in file:
                #I use an rstrip here because some of the files have trailing blank spaces
                jsonData.append(json.loads(line.rstrip()))
        
        df = pd.DataFrame.from_dict(jsonData)
        
        csvFileName = fileName[:len(fileName)-5] + '.csv'
        
        df.to_csv(directory + csvFileName)
        print('{0} created'.format(csvFileName))
        
        
        if createSample:
            np.random.seed(9001)
            msk = np.random.rand(len(df)) <= 0.1
            sample = df[msk]
            
            csvSampleFileName = fileName[:len(fileName)-5] + '_sample.csv'
            
            sample.to_csv(directory + csvSampleFileName)
            print('{0} created'.format(csvSampleFileName))
        
    print('This function took {} minutes to run'.format((time.time()-start)/60))
    

In [ ]:
fileNameList = ['user.json',
                'business.json']

json_to_csv('data/', fileNameList, createSample=True)

user.json opened
user.csv created
user_sample.csv created
business.json opened
business.csv created
business_sample.csv created
review.json opened


#### Keeping only "restaurant" business data and other associated data on users and reviews

In [4]:
def find_ftres_with_nan(df):
    all_nan = df.columns[df.isnull().all()].tolist()
    some_nan = df.columns[df.isnull().any()].tolist()
    print("All NaN Features: ", len(all_nan), all_nan, "Some NaN Features: ", len(some_nan), some_nan)
    return all_nan, some_nan

In [21]:
business['categories']

1.0

In [ ]:
business

In [19]:
business.columns

Index(['address', 'attributes', 'average_stars', 'business_id', 'categories',
       'city', 'compliment_cool', 'compliment_cute', 'compliment_funny',
       'compliment_hot', 'compliment_list', 'compliment_more',
       'compliment_note', 'compliment_photos', 'compliment_plain',
       'compliment_profile', 'compliment_writer', 'cool', 'elite', 'fans',
       'friends', 'funny', 'hours', 'is_open', 'latitude', 'longitude', 'name',
       'postal_code', 'review_count', 'stars', 'state', 'useful', 'user_id',
       'yelping_since'],
      dtype='object')

In [5]:
business = pd.read_csv('data/business.csv',encoding = "ISO-8859-1",index_col=0, nrows = 10000)
all_nan, some_nan = find_ftres_with_nan(business)

All NaN Features:  12 ['address', 'attributes', 'business_id', 'categories', 'city', 'hours', 'is_open', 'latitude', 'longitude', 'postal_code', 'stars', 'state'] Some NaN Features:  13 ['address', 'attributes', 'business_id', 'categories', 'city', 'elite', 'hours', 'is_open', 'latitude', 'longitude', 'postal_code', 'stars', 'state']


In [15]:
business['categories']

0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
5      NaN
6      NaN
7      NaN
8      NaN
9      NaN
10     NaN
11     NaN
12     NaN
13     NaN
14     NaN
15     NaN
16     NaN
17     NaN
18     NaN
19     NaN
20     NaN
21     NaN
22     NaN
23     NaN
24     NaN
25     NaN
26     NaN
27     NaN
28     NaN
29     NaN
        ..
9970   NaN
9971   NaN
9972   NaN
9973   NaN
9974   NaN
9975   NaN
9976   NaN
9977   NaN
9978   NaN
9979   NaN
9980   NaN
9981   NaN
9982   NaN
9983   NaN
9984   NaN
9985   NaN
9986   NaN
9987   NaN
9988   NaN
9989   NaN
9990   NaN
9991   NaN
9992   NaN
9993   NaN
9994   NaN
9995   NaN
9996   NaN
9997   NaN
9998   NaN
9999   NaN
Name: categories, Length: 10000, dtype: float64

In [10]:
# create a mask for restaurants
mask_restaurants = business['categories'].str.contains('Restaurants')

# create a mask for food
mask_food = business['categories'].str.contains('Food')

# apply both masks
restaurants_and_food = business[mask_restaurants & mask_food]

# number of businesses that have food and restaurant in their category
restaurants_and_food['categories'].count()

AttributeError: Can only use .str accessor with string values, which use np.object_ dtype in pandas

Even after taking buisnesses that have both food and restaurant in their categories, there are still irrelevant business categories in the data.

In [57]:
# an example row
restaurants_and_food.head(1)['categories'].values

array(['Specialty Food, Restaurants, Dim Sum, Imported Food, Food, Chinese, Ethnic Food, Seafood'],
      dtype=object)

In [58]:
categoryDF = restaurants_and_food['categories'].apply(lambda x: x[1:-1].split(',')).apply(pd.Series)
uniqueCategories = pd.DataFrame(categoryDF.stack().str.strip().unique())

In [59]:
categoriesToRemove = ['Grocery','Drugstores','Convenience Stores','Beauty & Spas','Photography Stores & Services',
                      'Cosmetics & Beauty Supply','Discount Store','Fashion','Department Stores','Gas Stations',
                      'Automotive','Music & Video','Event Planning & Services','Mobile Phones','Health & Medical',
                      'Weight Loss Centers','Home & Garden','Kitchen & Bath','Jewelry',"Children's Clothing",
                      'Accessories','Home Decor','Bus Tours','Auto Glass Services','Auto Detailing',
                      'Oil Change Stations', 'Auto Repair','Body Shops','Car Window Tinting','Car Wash',
                      'Gluten-Free','Fitness & Instruction','Nurseries & Gardening','Wedding Planning',
                      'Embroidery & Crochet','Dance Schools','Performing Arts',
                      'Wholesale Stores','Tobacco Shops','Nutritionists','Hobby Shops','Pet Services',
                      'Electronics','Plumbing','Gyms','Yoga','Walking Tours','Toy Stores','Pet Stores',
                      'Pet Groomers','Vape Shops','Head Shops',
                      'Souvenir Shops','Pharmacy','Appliances & Repair','Wholesalers','Party Equipment Rentals',
                      'Tattoo','Funeral Services & Cemeteries','Sporting Goods','Dog Walkers',
                      'Pet Boarding/Pet Sitting','Scavenger Hunts','Contractors','Trainers', 
                      'Customized Merchandise', 'Dry Cleaning & Laundry', 'Art Galleries'
                      'Tax Law', 'Bankruptcy Law', 'Tax Services', 'Estate Planning Law', 
                      'Business Consulting', 'Lawyers', 'Pet Adoption', 'Escape Games', 
                      'Animal Shelters', 'Commercial Real Estate', 'Real Estate Agents', 
                      'Real Estate Services', 'Home Inspectors']

In [60]:
restaurants_df = restaurants_and_food[~restaurants_and_food['categories'].str.contains('|'.join(categoriesToRemove))]


In [61]:
restaurants_df.to_csv('data/restaurants.csv')
restaurants_df = pd.read_csv('data/restaurants.csv', encoding='ISO-8859-1', index_col=0)
all_nan, some_nan = find_ftres_with_nan(restaurants_df)

All NaN Features:  20 ['average_stars', 'compliment_cool', 'compliment_cute', 'compliment_funny', 'compliment_hot', 'compliment_list', 'compliment_more', 'compliment_note', 'compliment_photos', 'compliment_plain', 'compliment_profile', 'compliment_writer', 'cool', 'elite', 'fans', 'friends', 'funny', 'useful', 'user_id', 'yelping_since'] Some NaN Features:  24 ['address', 'attributes', 'average_stars', 'compliment_cool', 'compliment_cute', 'compliment_funny', 'compliment_hot', 'compliment_list', 'compliment_more', 'compliment_note', 'compliment_photos', 'compliment_plain', 'compliment_profile', 'compliment_writer', 'cool', 'elite', 'fans', 'friends', 'funny', 'hours', 'postal_code', 'useful', 'user_id', 'yelping_since']


### Expanding business attributes & categories
After cleaning the business dataset, we expanded the attributes into boolean features. The attributes were a collection of a string of a dictionary. We evaluated this string as a dictionary and applied pd.Series across the result, which expanded this dictionary into multiple Boolean columns, as well as a few more dictionary columns. All remaining dictionary columns were manipulated in the same manner, leaving us ~70 Boolean columns. The business dataset also had a variable, categories, that was a string of a list. We expanded this list into separate columns for each category. Lastly, we then reduced our user dataset by filtering to users that existed in the reduced review dataset. The following functions were used for this process.

In [69]:
# #create_attributes

# #takes a dataframe as an input, as well as a list of columns that are dictionaries
# #takes each column that is a dictionary, and expands it into a series of dummy columns

# def create_attributes(df, dictList):
    
#     for dictionaryColumn in dictList:
        
#         #the attributes column is a string of dictionaries, so one extra step is taken to convert
#         if dictionaryColumn == 'attributes':
#             expandedColumns = df[dictionaryColumn].map(eval).apply(pd.Series)
#         else:
#             expandedColumns = df[dictionaryColumn].apply(pd.Series)
        
#         df = pd.concat([df.drop(dictionaryColumn,axis=1), 
#                    expandedColumns]
#                   ,axis=1)
        
#         #df.fillna(value='{}',inplace=True)
        
#     return df

In [70]:
# def expand_categories(df, cat_var, key):
#     all_cats = df[cat_var].str.cat(sep=', ')
#     all_cats = all_cats.replace('[', '')
#     all_cats = all_cats.replace(']', '')
#     all_cats = all_cats.replace("\'","")
#     all_cats = all_cats.replace('"','')
#     all_cats_list = all_cats.split(', ')
#     unique_cats = list(set(all_cats_list))
#     unique_cats.remove('Restaurants')
#     unique_cats.remove('Food')
#     df_cats = pd.DataFrame(index=df[key], columns=unique_cats, data=False)
#     df_out = df.merge(df_cats, how='left', left_on=key, right_index=True)
#     for cat in unique_cats:
#         df_out[cat] = df_out[cat_var].str.contains(cat)
#     return df_out

In [ ]:
# dictList = ['attributes', 'Ambience', 'BestNights', 'BusinessParking','Music','GoodForMeal']
# expanded = create_attributes(restaurants_df[['business_id','attributes']], dictList)
# dropColumns = expanded.columns.get_loc(0)
# keepColumns = list(compress(expanded.columns, ~dropColumns))
# expanded = expanded[keepColumns]

In [72]:
restaurants_df.head(1)


,address,attributes,average_stars,business_id,categories,city,compliment_cool,compliment_cute,compliment_funny,compliment_hot,...,latitude,longitude,name,postal_code,review_count,stars,state,useful,user_id,yelping_since
1637139,30 Eglinton Avenue W,"{'RestaurantsReservations': 'True', 'GoodForMe...",NaN,QXAEGFB4oINsVuTFxEYKFQ,"Specialty Food, Restaurants, Dim Sum, Imported...",Mississauga,NaN,NaN,NaN,NaN,...,43.605499,-79.652289,Emerald Chinese Restaurant,L5R 3E7,128,2.5,ON,NaN,NaN,NaN


In [ ]:
# expanded.head(1)


In [74]:
# def reduce_review(df, business_list):
    
#     #drop columns where business_id or user_id is null
#     df.dropna(subset=['business_id','user_id'], how='any',inplace=True)
    
#     #restrict to businesses that are restaurants
#     df = df[df['business_id'].isin(business_list)]
    
#     #only keep user_id's with more than one review
#     df = df[df.groupby('user_id').user_id.transform(len) > 1]
    
#     #verify this worked by taking the minimum amount of user_id counts
#     print('The minimum amount of reviews per user is {}'
#           .format(np.min(df.groupby('user_id')['business_id'].count())))
#     return df

In [ ]:
# review = pd.read_csv('data/review.csv',encoding = "ISO-8859-1",index_col=0)
# restaurant_reviews = reduce_review(review, restaurants['business_id']) #create dataframe
# _, _ = find_ftres_with_nan(restaurant_reviews) #report if there are null values

# restaurant_reviews['review_date'] = pd.to_datetime(restaurant_reviews['date'])
# restaurant_reviews['review_year'] = restaurant_reviews['review_date'].dt.year
# restaurant_reviews['review_month'] = restaurant_reviews['review_date'].dt.month
# restaurant_reviews['review_weekday'] = restaurant_reviews['review_date'].dt.weekday_name

# rename_cols = {'cool': 'review_cool','funny':'review_funny','stars':'review_stars','useful':'review_useful'}
# restaurant_reviews.rename(columns=rename_cols, inplace=True)
# review_cols_to_drop = ['text', 'review_date', 'date']
# restaurant_reviews.drop(review_cols_to_drop, axis=1, inplace=True)

In [ ]:
# restaurant_reviews.head()


In [ ]:
# restaurant_reviews.to_csv('data/restaurant_reviews_final.csv')


In [ ]:
users_df = pd.read_csv('data/user.csv',encoding = "ISO-8859-1",index_col=0)
users_w_reviews = users_df[users_df['user_id'].isin(restaurant_reviews['user_id'])].copy()

list_user_vars = ['average_stars', 'compliment_cool', 'compliment_cute', 'compliment_funny', 
                  'compliment_hot', 'compliment_list', 'compliment_more', 'compliment_note', 
                  'compliment_photos', 'compliment_plain', 'compliment_profile', 
                  'compliment_writer', 'cool', 'elite', 'fans', 'friends', 'funny', 
                  'name', 'review_count', 'useful']
vars_to_rename_users = dict(zip(list_user_vars, ['user_' + var for var in list_user_vars]))
users_w_reviews = users_w_reviews.rename(columns=vars_to_rename_users)

users_w_reviews['yelping_since'] = pd.to_datetime(users_w_reviews['yelping_since'])
users_w_reviews['yelping_since'] = users_w_reviews['yelping_since'].dt.year

users_w_reviews['user_elite_flag'] = users_w_reviews['user_elite'].apply(len) != 2
users_w_reviews['user_friends_flag'] = users_w_reviews['user_friends'].apply(len) != 2

users_cols_to_drop = ['user_elite', 'user_friends', 'user_name']
users_w_reviews.drop(users_cols_to_drop, axis=1, inplace=True)

users_w_reviews.to_csv('data/user_final.csv')